# Experiment 2:

- Our base sketch transformer architecture + class embedding layer
- Multi-class Conditional Model
- DeltaPenPositionTokenizer

In [1]:
from experiment_dir import set_cwd_project_root

set_cwd_project_root()

In [ ]:
import torch

torch.cuda.empty_cache()

In [ ]:
from dataset import QuickDrawDataset
from sketch_tokenizers import DeltaPenPositionTokenizer
from models import SketchTransformerConditional
from runner import SketchTrainer, sample
from prepare_data import stroke_to_bezier_single, clean_svg

label_names = ["cat", "dog", "bird", "fish", "crab", "guitar", "sailboat", "bulldozer", "apple", "monkey", "skull", "whale"]
dataset = QuickDrawDataset(label_names=label_names, download=True)
tokenizer = DeltaPenPositionTokenizer(bins=32)

model = SketchTransformerConditional(
    vocab_size=len(tokenizer.vocab),
    d_model=768,
    nhead=12,
    num_layers=16,
    max_len=200,
    num_classes=len(label_names),
)

training_config = {
    "batch_size": 128,
    "num_epochs": 15,
    "learning_rate": 1e-4,
    "log_dir": "logs/sketch_transformer_experiment_3",
    "splits": [0.85, 0.1, 0.05],
    # "use_padding_mask": True,
    # "checkpoint_path": "logs/sketch_transformer_experiment_3/20251112_125146/model_3.pt",
}

trainer = SketchTrainer(model, dataset, tokenizer, training_config)

2025-11-12 13:43:44.550112: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-12 13:43:44.590414: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-12 13:43:45.633471: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Using device: cuda


Loading QuickDrawDataset: 8it [00:00, 347.06it/s]
/student/taylo831/.local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Tokenizing dataset: 100%|██████████| 8/8 [00:00<00:00, 1765.84it/s]


Resumed training from checkpoint: logs/sketch_transformer_experiment_3/20251112_125146/model_3.pt at epoch 3


Initial Eval: 100%|██████████| 824/824 [00:02<00:00, 334.49it/s]


In [5]:
trainer.train_mixed(training_config["num_epochs"])

Epoch 4/15 [train]:   3%|▎         | 219/6996 [00:26<13:44,  8.22it/s]


KeyboardInterrupt: 

In [ ]:
generations_inline = ""
generations = []

for j, label_name in enumerate(label_names):
    for i in range(5):
        generated = sample(
            model=trainer.model,
            start_tokens=[trainer.tokenizer.vocab["START"]],
            temperature=0.8,
            top_k=20,
            top_p=0.7,
            greedy=False,
            eos_id=trainer.tokenizer.vocab["END"],
            class_label=j,
        )
        decoded_sketch = tokenizer.decode(generated, stroke_width=0.3)
        decoded_sketch = stroke_to_bezier_single(decoded_sketch)
        decoded_sketch = clean_svg(decoded_sketch)

        generations_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated {label_name}</b><br>{decoded_sketch}</div>'
        generations.append((generated, decoded_sketch))


from IPython.display import HTML, display

display(HTML(generations_inline))